The Core, the heart, the soul.

Experimental

In [3]:
# Check if cuda is enabled 
import torch
torch.cuda.is_available()


True

In [4]:
# Model location: './model/models
# Cache path code: cache_dir='.model/models'

from transformers import AutoTokenizer, AutoModelForCausalLM
from airllm import AutoModel



'''
# Loading model(first step, downloading model)
tokenizer = AutoTokenizer.from_pretrained("yunconglong/Truthful_DPO_TomGrc_FusionNet_7Bx2_MoE_13B", cache_dir="./model")
model = AutoModelForCausalLM.from_pretrained("yunconglong/Truthful_DPO_TomGrc_FusionNet_7Bx2_MoE_13B", cache_dir="./model")
'''


MMAX_LENGTH = 128

# Load the model outside of the loop
model = AutoModel.from_pretrained("./model/models--yunconglong--Truthful_DPO_TomGrc_FusionNet_7Bx2_MoE_13B/snapshots/b780b7605afbb7417a0054716215ce90ca03b299")
# Set the attention mechanism implementation to SDPA
model.config.attention_impl = "sdpa"

# Now you can use the model repeatedly to process different input texts
while True:
    input_text = input("Enter text (or 'exit' to quit): ")
    if input_text.lower() == "exit":
        break

    # Tokenize the input text
    input_tokens = model.tokenizer(input_text,
                                    return_tensors="pt", 
                                    return_attention_mask=False, 
                                    truncation=True, 
                                    max_length=MAX_LENGTH, 
                                    padding=False)
    
    # Generate output
    generation_output = model.generate(input_tokens['input_ids'].cuda(), 
                                        max_new_tokens=20,
                                        use_cache=True,
                                        return_dict_in_generate=True)

    # Decode and print the output
    output = model.tokenizer.decode(generation_output.sequences[0])
    print("Output:", output)


found index file...


  0%|          | 0/35 [00:00<?, ?it/s]

Loading shard 1/6


  3%|▎         | 1/35 [00:03<01:47,  3.17s/it]

saved as: model/models--yunconglong--Truthful_DPO_TomGrc_FusionNet_7Bx2_MoE_13B/snapshots/b780b7605afbb7417a0054716215ce90ca03b299/splitted_model/model.embed_tokens.safetensors


...

either BetterTransformer or attn_implementation='sdpa' is available, creating model directly


running layers(self.running_device): 100%|██████████| 35/35 [07:57<00:00, 13.63s/it]


either BetterTransformer or attn_implementation='sdpa' is available, creating model directly


running layers(self.running_device):  31%|███▏      | 11/35 [02:28<05:23, 13.48s/it]


KeyboardInterrupt: 